In [1]:
import pandas as pd
import numpy as np
import tqdm
import warnings
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score

pd.set_option("max_rows", 200)
pd.set_option("max_columns", 50)
tqdm.tqdm.pandas()
%matplotlib inline

In [2]:
df = pd.read_pickle("../input/riiid-test-answer-prediction/split10/train_0.pickle").sort_values(["user_id", "timestamp"]).reset_index(drop=True)

In [3]:
import sys
sys.path.append("../")
# df = pd.read_pickle("../input/riiid-test-answer-prediction/split10/train_0.pickle")
df = df.sort_values(["user_id", "timestamp"])
# large_user_id = df["user_id"].value_counts()
# large_user_id = large_user_id[large_user_id > 1000] 
# df = df[df["user_id"].isin(large_user_id.index)]
# df = df[df["answered_correctly"] != -1]

In [4]:
df["answered_correctly"] = df["answered_correctly"].replace(-1, np.nan)

In [6]:
df["session"] = df["timestamp"] - df.groupby("user_id")["timestamp"].shift(1)
df["session"] = (df["session"] > 1000000).astype(int)
df["session"] = df.groupby("user_id")["session"].cumsum()

In [10]:
df.head(10000).groupby(["user_id", "session"])["answered_correctly"].describe().head(200)

count      mean       std  min   25%  50%   75%  max
user_id session                                                      
24600   0         50.0  0.340000  0.478518  0.0  0.00  0.0  1.00  1.0
108310  0         37.0  0.756757  0.434959  0.0  1.00  1.0  1.00  1.0
138650  0         40.0  0.700000  0.464095  0.0  0.00  1.0  1.00  1.0
        1         66.0  0.742424  0.440650  0.0  0.25  1.0  1.00  1.0
        2         36.0  0.555556  0.503953  0.0  0.00  1.0  1.00  1.0
        3         15.0  0.866667  0.351866  0.0  1.00  1.0  1.00  1.0
        4         12.0  0.750000  0.452267  0.0  0.75  1.0  1.00  1.0
        5         14.0  0.785714  0.425815  0.0  1.00  1.0  1.00  1.0
        6         34.0  0.500000  0.507519  0.0  0.00  0.5  1.00  1.0
        7          1.0  1.000000       NaN  1.0  1.00  1.0  1.00  1.0
        8          6.0  1.000000  0.000000  1.0  1.00  1.0  1.00  1.0
        9          1.0  1.000000       NaN  1.0  1.00  1.0  1.00  1.0
        10        19.0  0.736842  0.452414  0.0  0.50  1.0  1.00  1.0
        11        25.0  0.840000  0.374166  0.0  1.00  1.0  1.00  1.0
        12         3.0  0.666667  0.577350  0.0  0.50  1.0  1.00  1.0
        13         9.0  0.666667  0.500000  0.0  0.00  1.0  1.00  1.0
        14        21.0  0.571429  0.507093  0.0  0.00  1.0  1.00  1.0
        15         1.0  1.000000       NaN  1.0  1.00  1.0  1.00  1.0
        16        10.0  0.700000  0.483046  0.0  0.25  1.0  1.00  1.0
        17         4.0  0.750000  0.500000  0.0  0.75  1.0  1.00  1.0
        18        12.0  0.833333  0.389249  0.0  1.00  1.0  1.00  1.0
        19        19.0  1.000000  0.000000  1.0  1.00  1.0  1.00  1.0
        20         7.0  1.000000  0.000000  1.0  1.00  1.0  1.00  1.0
        21         8.0  0.375000  0.517549  0.0  0.00  0.0  1.00  1.0
        22         1.0  1.000000       NaN  1.0  1.00  1.0  1.00  1.0
        23        32.0  0.718750  0.456803  0.0  0.00  1.0  1.00  1.0
        24         5.0  1.000000  0.000000  1.0  1.00  1.0  1.00  1.0
        25        18.0  0.833333  0.383482  0.0  1.00  1.0  1.00  1.0
        26         1.0  1.000000       NaN  1.0  1.00  1.0  1.00  1.0
        27         4.0  1.000000  0.000000  1.0  1.00  1.0  1.00  1.0
        28        15.0  0.600000  0.507093  0.0  0.00  1.0  1.00  1.0
        29        20.0  0.750000  0.444262  0.0  0.75  1.0  1.00  1.0
        30        20.0  0.750000  0.444262  0.0  0.75  1.0  1.00  1.0
        31         1.0  1.000000       NaN  1.0  1.00  1.0  1.00  1.0
        32        22.0  0.772727  0.428932  0.0  1.00  1.0  1.00  1.0
        33         9.0  0.333333  0.500000  0.0  0.00  0.0  1.00  1.0
        34        12.0  0.833333  0.389249  0.0  1.00  1.0  1.00  1.0
        35         4.0  0.500000  0.577350  0.0  0.00  0.5  1.00  1.0
        36         8.0  0.625000  0.517549  0.0  0.00  1.0  1.00  1.0
        37        48.0  0.750000  0.437595  0.0  0.75  1.0  1.00  1.0
        38         1.0  1.000000       NaN  1.0  1.00  1.0  1.00  1.0
        39        19.0  0.947368  0.229416  0.0  1.00  1.0  1.00  1.0
        40         9.0  0.666667  0.500000  0.0  0.00  1.0  1.00  1.0
        41        14.0  0.571429  0.513553  0.0  0.00  1.0  1.00  1.0
        42         1.0  1.000000       NaN  1.0  1.00  1.0  1.00  1.0
        43         1.0  1.000000       NaN  1.0  1.00  1.0  1.00  1.0
        44        22.0  0.636364  0.492366  0.0  0.00  1.0  1.00  1.0
        45        22.0  0.818182  0.394771  0.0  1.00  1.0  1.00  1.0
        46         1.0  1.000000       NaN  1.0  1.00  1.0  1.00  1.0
        47         1.0  0.000000       NaN  0.0  0.00  0.0  0.00  0.0
        48         4.0  0.750000  0.500000  0.0  0.75  1.0  1.00  1.0
        49         1.0  1.000000       NaN  1.0  1.00  1.0  1.00  1.0
        50         8.0  0.750000  0.462910  0.0  0.75  1.0  1.00  1.0
        51        23.0  0.565217  0.506870  0.0  0.00  1.0  1.00  1.0
        52        15.0  0.866667  0.351866  0.0  1.00  1.0  1.00  1.0
        53         3.0  1.0